# Computer Vision (911.908)

## <font color='crimson'>Automatic differentiation (aka AutoGrad)</font>

**Changelog**:
- *Sep. 2020*: initial version (using PyTorch v1.6) 
- *Sep. 2021*: adaptations to PyTorch v1.9
- *Oct. 2022*: adaptations to PyTorch v1.12.1 + minor fixes

---

In this lecture, we take a look at the mechanics of **automatic differentiation** (aka *AutoGrad* / *AutoDiff*) in PyTorch. These techniques are at the very heart of today's frameworks for learning with neural networks.

You can also find the official (tutorial-like) documentation [here](https://pytorch.org/docs/stable/notes/autograd.html).

---

## Contents

- [Introductory example](#Introductory-example)
- [A  slightly more involved example](#A-slightly-more-involved-example)
- [The AutoGrad machinery](#The-AutoGrad-machinery)

---

In [1]:
#
# Running this code requires torchviz. Install via
#
# pip install torchviz
#

%load_ext autoreload
%autoreload 2

import sys
import agtree2dot

sys.path.append(".")

from utils import visualize_DAG
from IPython.display import HTML
from torchviz import make_dot


---

## Introductory example

In this example, we take the following function

$$f(x;w,b) = \max\{wx + b, 0\} = \text{ReLU}(wx+b)$$

where 

$$\text{ReLU}: x \mapsto \text{ReLU}(x) = \max\{x, 0\}\enspace.$$

We first look at its representation as a **computation graph** (with example values for $w,b$ and $x$ added):

<img src="DAG0.svg" alt="drawing" width="700"/>

Lets compute  

$$ \frac{\partial a}{\partial b} = \frac{\partial a}{\partial v}\frac{\partial v}{\partial b}$$

First, 

$$ \frac{\partial v}{\partial b} = 1 $$

and (using a sub-gradient)

$$ \frac{\partial a}{\partial v} = \frac{\partial}{\partial v} \text{ReLU}(v) = 
\begin{cases}
0 & \text{if}~v \leq 0\\
1 & \text{else}
\end{cases}\enspace.
$$

Hence, in our case, we have (as $v=7$)

$$\frac{\partial a}{\partial v} = 1$$

and consequently 

$$
\frac{\partial a}{\partial b} = 1\enspace.
$$

Let's try to compute the partial derivative of $a$ wrt. $w$:

$$
\frac{\partial a}{\partial w} = \frac{\partial a}{\partial v}\frac{\partial v}{\partial u}\frac{\partial u}{\partial w}
$$

We have, 

$$
\frac{\partial v}{\partial u} = 1
$$

and 

$$
\frac{\partial u}{\partial w} = x
$$

Combined, we obtain (if we use the fact that $x=3$)

$$
\frac{\partial a}{\partial w} = \frac{\partial a}{\partial v}\frac{\partial v}{\partial u}\frac{\partial u}{\partial w} = 1\cdot 1\cdot 3 = 3
$$



Conceptually, the **forward pass** (i.e., the computation of the function value) is a sequence of standard tensor computations (i.e., in this example, simply the computation of all intermediate results). We need the **Directed Ascyclic Graph (DAG)** of tensor operations only to compute derivatives (via the chain rule). 

When executing tensor operations, PyTorch can automatically (on-the-fly) construct the graph of operations to compute the gradient of any quantity with respect to any tensor involved.

In [2]:
import torch
from torch.autograd import grad
import torch.nn.functional as F # functional API

**Example**

Let's implement the simple example from above. Note the use of `requires_grad=True` (`False` by default) for the variable `w` and `b` (explained below).

In [3]:
x = torch.tensor([3.])
w = torch.tensor([2.], requires_grad=True) 
b = torch.tensor([1.], requires_grad=True) 

u = x*w
v = u+b
a = torch.clamp(v, min=0) # alternatively: a = F.relu(x*w + b)
print(a)

tensor([7.], grad_fn=<ClampBackward1>)


A Tensor has a Boolean field `requires_grad`, set to `False` by default, which
states if PyTorch should build the graph of operations so that gradients with
respect to it can be computed.

**Note**: Only *floating point type* tensors can have their gradient computed.

In [4]:
print("v =", v.item())

v = 7.0


Compute gradient $\partial a/\partial w$

In [5]:
grad_w = grad(a, w, retain_graph=True)
print(grad_w[0].item())

3.0


Compute gradient $\partial a/\partial b$

In [6]:
print(grad(a, b)[0].item())

1.0


Note the use of `retain_graph=True` earlier. This is not required here, but in case you want to execute that cell multiple time, you will need it. The reason is that, by default, the part of the graph that computes `a` is destroyed (for memory reasons) once the gradient computation is done. If `retain_graph=False`, then execution of that statement will fail (unless you do another forward pass). 

---

## A slightly more involved example

In this example, we have a slightly more complex computation graph with a *shared* weight $w$ and two paths to arrive at the result. The function is

$$f(x;w) = (wx)^2 + (wx)^3$$

<img src="DAG1.svg" alt="drawing" width="700"/>

Let's compute the partial derivative of $a$ wrt. $w$ by hand once more:


$$
\frac{\partial a}{\partial w} = \frac{\partial a}{\partial v_1}\frac{\partial v_1}{\partial u_1}\frac{\partial u_1}{\partial w} + \frac{\partial a}{\partial v_2}\frac{\partial v_2}{\partial u_2}\frac{\partial u_2}{\partial w} = 
1\cdot 2u_1\cdot x + 1 \cdot 3u_2^2 \cdot x = 8 + 24 = 32
$$

Again, having **all the intermediate values** during the **forward pass** makes it very easy to actually compute the gradient of $a$ wrt. $w$, i.e., $\partial a/\partial w$.



In [7]:
x = torch.tensor([2.])
w = torch.tensor([1.], requires_grad=True)

u1 = x*w
u2 = x*w

v1 = torch.pow(u1, 2)
v2 = torch.pow(u2, 3)

a = v1 + v2
print("a =", a.item())

a = 12.0


Next, we compute the gradient of $a$ wrt. $w$, i.e., $\partial a/\partial w$

In [8]:
print(grad(a, w)[0].item())

32.0


---

## The AutoGrad machinery

The autograd DAG is encoded through the fields `grad_fn` of Tensors, and the
fields `next_functions` of functions.

In [9]:
# below is another way of saying requires_grad=True
x = torch.tensor([ 1.0, -2.0, 3.0, -4.0 ]).requires_grad_() 
a = x.pow(2.)
s = a.sum()
print(s.item())

30.0


Below, we visualize a simple computation graph:

In [10]:
x = torch.tensor([1., 2., 2.]).requires_grad_()
q = x.norm() # L2-norm of tensor sqrt(1^2+2^2+2^2)=3
print(q.item())

3.0


In [11]:
agtree2dot.save_dot(q, {x: 'x', q: 'q'}, open('simple1.dot', 'w'))
HTML(visualize_DAG('simple1.dot', 'simple1.jpg'))

**Example**

Here's another, slightly more involved, example: We have $\mathbf{x} \in \mathbb{R}^{10}$ and **first** compute (noting that $\text{tanh}$ is applied componentwise).

$$
\mathbf{h}  = \text{tanh}(\mathbf{W}_1\mathbf{x} + \mathbf{b}_1) \\
$$

with $\mathbf{W}_1 \in \mathbb{R}^{20 \times 10}$ and $\mathbf{b}_1 \in \mathbb{R}^{20}$. This results in $\mathbf{h} \in \mathbb{R}^{20}$. **Second**, we compute

$$
\hat{\mathbf{y}}  = \text{tanh}(\mathbf{W}_2\mathbf{h} + \mathbf{b}_2)
$$

with $\mathbf{W}_2 \in \mathbb{R}^{5 \times 20}$ and $\mathbf{b}_2 \in \mathbb{R}^5$. This results in 
$\hat{\mathbf{y}} \in \mathbb{R}^5$. And **finally**, we compute (the mean-squared error)

$$
L(\mathbf{y},\hat{\mathbf{y}}) = \frac{1}{5} \sum_{i=1}^5(y_i-\hat{y}_i)^2
$$

where $y_i,\hat{y}_i$ are the $i$-th elements of $\mathbf{y}$ and $\hat{\mathbf{y}}$, respectively. In the code below, $y_i$ (which we use as targets) are just random numbers.

In [17]:
w1 = torch.rand(20, 10).requires_grad_()
b1 = torch.rand(20).requires_grad_()
w2 = torch.rand(5, 20).requires_grad_()
b2 = torch.rand(5).requires_grad_()

x = torch.rand(10)
h = torch.tanh(w1 @ x + b1)
yhat = torch.tanh(w2 @ h + b2)

y = torch.rand(5)

loss = (y - yhat).pow(2).mean()
loss.backward() # computes the partial derivatives wrt. w1,b1,w2,b2
print(loss.item())

0.2811025083065033


Lets look at the computation graph ...

In [18]:
agtree2dot.save_dot(loss, 
                    {
                        w1: 'w1',
                        b1: 'b1',
                        w2: 'w2',
                        b2: 'b2',
                        loss: 'loss'
                    }, open('simple2.dot', 'w'))
HTML(visualize_DAG('simple2.dot', 'simple2.jpg'))

### Using `torch.no_grad()`

The `torch.no_grad()` **context** switches off the autograd machinery, and can be
used for operations such as parameter updates.

In [19]:
a = torch.tensor( 0.7).requires_grad_()
b = torch.tensor(-0.7).requires_grad_()
eta = 0.1

for k in range(100):
    l = (a - 1)**2 + (b + 1)**2 + (a - b)**2 # i.e., the forward pass
    ga, gb = torch.autograd.grad(l, (a, b))

    # walk towards the direction of the negative gradient 
    # (aka gradient descend) - both statements will not change
    # the computation graph, because they are within the 
    # torhc.no_grad context!
    with torch.no_grad():
        a -= eta * ga 
        b -= eta * gb
        
print('%.06f' % a.item(), '%.06f' % b.item())

0.333333 -0.333333


In case you do not believe that this is the minimum, fire up Mathematica and run the `2DMinimumExample.nb` available in this directory :)

### Using `detach()`

The `detach()` method creates a tensor which shares the data, but does not
require gradient computation, and is not connected to the current computation graph.

This method should be used when the gradient should not be propagated
beyond a variable, or to update leaf tensors.

### A simple example

In [25]:
x = torch.ones(10, dtype=torch.float32, requires_grad=True)

y = x**2
z = x**3
r=(y+z).sum()
print(r.item())
r.backward() # computes the gradient of r wrt. x
print(x.grad)

20.0
tensor([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])


Now, lets exclude the contribution of the $x^3$ branch via `detach()`.

In [26]:
x = torch.ones(10, dtype=torch.float32, requires_grad=True)

y = x**2
z = (x**3).detach()
r=(y+z).sum()
print(r.item())
r.backward()
print(x.grad)

20.0
tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])


Getting back to our exampe of finding the minimum of a function (see above) ...

In [27]:
a = torch.tensor( 0.5).requires_grad_()
b = torch.tensor(-0.5).requires_grad_()
eta = 0.1

for k in range(100):
    l = (a - 1)**2 + (b + 1)**2 + (a.detach() - b)**2
    ga, gb = torch.autograd.grad(l, (a, b))
    
    with torch.no_grad():
        a -= eta * ga
        b -= eta * gb
        
print('%.06f' % a.item(), '%.06f' % b.item())

1.000000 -0.000000
